In [1]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [2]:
sys.path

['/root/HSE_diploma/ipynb',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/root/HSE_diploma/',
 '/root/HSE_diploma/evaluator/']

In [3]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model 
from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


/root/HSE_diploma/prompter/templates/


In [4]:

def init_lora_model_and_tokenizer(default_model,
                             LORA_R,
                             LORA_ALPHA,
                             LORA_DROPOUT
                            ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer



class MyCustomCallback(TensorBoardCallback):
    #log_bleu_steps_factor = 5
    bleu_generation_max_new_tokens = 30
    bleu_fn_test_data = "/root/data/t2c_answers.json"
    bleu_fn_etalon = "/root/data/answers.json"
    log_step = 0
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        super().on_log(args, state, control, logs=logs, **kwargs)
        #print("kwargs", len(kwargs), kwargs.keys())
        if self.tb_writer is not None:
            #print(state)
            #print(state.global_step)
            #print(self.log_step)
            if (self.log_step % self.log_bleu_steps_factor ==0):
                model = kwargs['model']
                tokenizer = kwargs['tokenizer']
                
                model.eval()
                assert not model.training
                generation_config = GenerationConfig(max_new_tokens = self.bleu_generation_max_new_tokens,
                                                     # min_new_tokens = 5,
                                                     temperature = 1.0
                                                    )
                print("generation_config:", generation_config)
                evaluator = EvaluateTestSet(generation_config = generation_config,
                                            fn_test_data = self.bleu_fn_test_data,
                                            fn_etalon = self.bleu_fn_etalon,
                                            batch_size = 1
                                       )

                metric_res = evaluator.evaluate(model=model, 
                                                tokenizer=tokenizer,
                                               )
                model.train()
                assert model.training
                print(metric_res)
                for key, val in metric_res.items():
                    #add "custom/something"
                    self.tb_writer.add_scalar(key, val, state.global_step)
                self.tb_writer.flush()
            self.log_step += 1

In [5]:
CONFIG_PATH = "/root/HSE_diploma/experiments_configs/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v36"
# t2c_concode_220428_v18.json

In [6]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [7]:
experiment_config['resume_from_checkpoint'] = False

In [8]:
assert experiment_config['resume_from_checkpoint'] == False

In [9]:
assert experiment_config['experiment_name'] == experiment_name

In [10]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [11]:
print("current_experiment_path", current_experiment_path)

current_experiment_path /root/experiments/t2c_concode_220428_v36


In [12]:
!mkdir {current_experiment_path}

mkdir: cannot create directory ‘/root/experiments/t2c_concode_220428_v36’: File exists


In [13]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [14]:
assert experiment_config["default_model"] == "google/flan-t5-base"

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained(experiment_config["default_model"])
model = T5ForConditionalGeneration.from_pretrained(experiment_config["default_model"], 
                                                   device_map="auto", 
                                                   #torch_dtype=torch.float16
                                                  )


In [16]:
experiment_config["fn_train_dataset"] = '/root/data/t2c_1000train.json'

In [17]:
data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-0178db4b5cbb5cd4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
# model, tokenizer = init_lora_model_and_tokenizer(default_model = experiment_config["default_model"],
#                                                  LORA_R = experiment_config["LORA_R"],
#                                                  LORA_ALPHA = experiment_config["LORA_ALPHA"],
#                                                  LORA_DROPOUT = experiment_config["LORA_DROPOUT"]
#                                                 )



In [19]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    if hasattr(model, "print_trainable_parameters"):
        model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

tokenizer.pad_token_id 0
tokenizer.eos_token_id 1
tokenizer.bos_token_id None
tokenizer.eos_token_id 1
model.config.pad_token_id 0
model.config.eos_token_id 1
model.config.bos_token_id None
model.config.eos_token_id 1


In [20]:
# experiment_config

In [21]:
from prompter import Prompter
prompter = Prompter()

def generate_prompt(data_point):
    if "input" in data_point and data_point["input"]:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        input = data_point["input"],
                                        label = data_point["output"]
                                       )
    else:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        #input = None,
                                        label = data_point["output"]
                                       )

/root/HSE_diploma/prompter/templates/


In [22]:
experiment_config["truncation"],\
experiment_config["CUTOFF_LEN"],\
experiment_config["padding"]

(True, 256, 'max_length')

In [23]:
# experiment_config['MICRO_BATCH_SIZE'] = 1
data['train'][0]

{'output': 'void function ( ) { int loc0 = 0 ; int loc1 = 9 ; int loc2 = 1 ; int loc3 = ResponseHeader . calculatePageNumber ( loc0 , loc1 , loc2 ) ; assertThat ( loc3 , is ( 1 ) ) ; }',
 'instruction': "Generate java code\ncan we tell when we 're at the start of the first page when there is only one page ? ",
 'input': ' PlaceHolder placeHolder \n\n void testCalculateTotalPages02 \n void testCalculatePageNumber01 \n void testCalculateTotalPages03 \n void testCalculatePageNumber03 \n void testCalculateTotalPages01 \n void testCalculatePageNumber04 \n void testCalculatePageNumber05 \n void testCalculateTotalPages04'}

In [24]:
def generateT5_input(dct, tokenizer):
    instruction  = dct["instruction"]
    output = dct["output"]
    input = dct["input"]

    prompt = f"{instruction}\n###Instruction{input}\n###Responce"
    result = f"{output}"
            
    prompts_ids = tokenizer(prompt, 
                                     truncation=experiment_config["truncation"],
                                     max_length=experiment_config["CUTOFF_LEN"],
                                     padding=experiment_config["padding"],
                                     return_tensors='pt'
                                   )
        
    results_ids = tokenizer(result, 
                         truncation=experiment_config["truncation"],
                         max_length=experiment_config["CUTOFF_LEN"],
                         padding=experiment_config["padding"],
                         return_tensors='pt'
                       )
    #input_ids,attention_mask,decoder_input_ids,decoder_attention_mask

    return {"input_ids" : prompts_ids['input_ids'],
            "attention_mask" : prompts_ids['attention_mask'],
            "decoder_input_ids" : results_ids['input_ids'],
            "decoder_attention_mask" :results_ids['attention_mask'] 
           }

In [100]:
# generateT5_input({"instruction": "i",
#                    "output": "O",
#                    "input": "123"}, tokenizer)

In [25]:
data = data.shuffle().map(
    lambda data_point: generateT5_input(data_point, tokenizer)
#         generate_prompt(data_point),
#         truncation=experiment_config["truncation"],
#         max_length=experiment_config["CUTOFF_LEN"],
#         padding=experiment_config["padding"]
#     )
)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [26]:
# data = data.remove_columns("input")

In [43]:
# data = data.remove_columns(["input", "output", "instruction"])

In [46]:
# # model(**o(data['train'][0:1]))

# # o = transformers.DataCollatorForSeq2Seq(tokenizer)
# # o(data['train'][0:2])
# o([data['train'][0], data['train'][0]])

In [42]:
# data['train'][0:2]['input_ids']

In [102]:
# data['train'][0]['input_ids']

In [44]:
# data['eval'][0]
# data['train'][0].keys()

In [75]:
# assert experiment_config["fp16"] == False
# tokenizer(['123'], return_tensors='pt')

{'input_ids': tensor([[    3, 14574,     1]]), 'attention_mask': tensor([[1, 1, 1]])}

In [108]:
# # def foo(tokenizer):
# #     print(1)
# #     return 

# class Foo:
#     def __init__(self, tokenizer):
#         self.tokenizer = tokenizer
        
#     def __call__(self, features, return_tensors=None):
#         prompts = []
#         results  = []
#         for text in features:
#             instruction  = text["instruction"]
#             output = text["output"]
#             input = text["input"]
            
#             prompt = f"{instruction}\n###Instruction{input}\n###Responce"
#             answer = f"{output}"
            
#             prompts.append(prompt)
#             results.append(answer)
        
#         prompts_ids = self.tokenizer(prompts, 
#                                      truncation=experiment_config["truncation"],
#                                      max_length=experiment_config["CUTOFF_LEN"],
#                                      padding=experiment_config["padding"],
#                                      return_tensors='pt'
#                                    )
        
#         results_ids = self.tokenizer(results, 
#                              truncation=experiment_config["truncation"],
#                              max_length=experiment_config["CUTOFF_LEN"],
#                              padding=experiment_config["padding"],
#                              return_tensors='pt'
#                            )
#         #input_ids,attention_mask,decoder_input_ids,decoder_attention_mask
            
#         return {"input_ids" : prompts_ids['input_ids'],
#                 "attention_mask" : prompts_ids['attention_mask'],
#                 "decoder_input_ids" : results_ids['input_ids'],
#                 "decoder_attention_mask" :results_ids['attention_mask'] 
#                }
# # o = transformers.DataCollatorForSeq2Seq(tokenizer)
# # o()

# # a = Foo(tokenizer)
# generateT5_input({"instruction": "i",
#                    "output": "O",
#                    "input": "123"}, tokenizer)

In [53]:
# # experiment_config["padding"],
# # experiment_config["truncation"],
# # data['train'][0]
# o = transformers.DataCollatorForSeq2Seq(tokenizer)
# a = {k:v for k,v in data['train'][0].items() if k in set(["input_ids", "attention_mask",
#                                                    "decoder_input_ids", "decoder_attention_mask"
#                                                  ]
#                                                 )
#     }

# # a
# # o([a, a])
# # o()
# # transformers.DataCollatorForSeq2Seq(tokenizer)(**data['train'][0])
# # transformers.DataCollatorForSeq2Seq()
class MyDataCollatorForSeq2Seq(transformers.DataCollatorForSeq2Seq):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __cal__(self, features, return_tensors=None):
        print(features)
        super().__call__(features = [features], return_tensors = return_tensors)
        
o = MyDataCollatorForSeq2Seq(tokenizer, 
                            )

print(o(data['train'][0]))
# # o(a)
# # o.tokenizer

KeyError: 0

In [50]:
data['train'][0]

{'input_ids': [[6939,
   2206,
   3,
   27578,
   1081,
   3,
   9,
   1006,
   1573,
   12,
   21830,
   80,
   7358,
   13,
   331,
   45,
   8,
   16826,
   38,
   3,
   9,
   720,
   10116,
   63,
   3,
   5,
   1713,
   30345,
   1570,
   7,
   26853,
   16,
   17,
   9400,
   16,
   17,
   6306,
   908,
   14120,
   16,
   17,
   3902,
   16,
   17,
   7358,
   134,
   1737,
   3,
   12186,
   356,
   17748,
   23,
   106,
   16,
   17,
   129,
   3845,
   2632,
   3,
   12186,
   356,
   16,
   17,
   6306,
   908,
   129,
   22481,
   2796,
   89,
   17,
   7638,
   279,
   155,
   16,
   17,
   65,
   107,
   22737,
   3,
   12840,
   109,
   152,
   129,
   3,
   12840,
   109,
   152,
   4081,
   7,
   3,
   12186,
   964,
   22341,
   12,
   11500,
   53,
   16,
   17,
   129,
   518,
   23,
   26,
   189,
   3,
   12186,
   10807,
   16,
   17,
   6306,
   908,
   129,
   26465,
   235,
   51,
   448,
   2632,
   7638,
   279,
   155,
   1713,
   30345,
   1649,
   7990,
 

In [185]:
# experiment_config['eval_steps'] = 1
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"]*2,
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"],
        load_best_model_at_end = experiment_config["load_best_model_at_end"]
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(tokenizer),
#     callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: too many values to unpack (expected 2)

In [28]:
model.save_pretrained(current_experiment_path)

In [31]:
s = generate_prompt({"input": "xui", "instruction": "123", "output": "ppp"})

In [48]:
# model(**tokenizer(s,
#                   truncation=experiment_config["truncation"],
#                   max_length=experiment_config["CUTOFF_LEN"],
#                   padding=experiment_config["padding"], 
#                   return_tensors='pt'
#                  )
#      )
model(input_ids=torch.Tensor([data['train'][0]['input_ids']]).long().to('cuda'),
      decoder_input_ids=torch.Tensor([data['train'][0]['input_ids']]).long().to('cuda'))

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1,No log,0.321902
2,No log,0.316308
3,No log,0.305711
4,No log,0.290590
5,No log,0.272077
6,No log,0.251525
7,No log,0.230091
8,No log,0.209516
9,No log,0.191129
10,0.614600,0.175229



KeyboardInterrupt



Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.179400,1.262794
200,1.115300,1.161276
300,0.991500,1.134106
400,1.003000,1.125500
500,0.930900,1.119192
600,0.972400,1.115012
700,1.053000,1.112176
800,0.965700,1.112523
900,0.982100,1.108011


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.29.2"
}



...

{'EM': 0.0, 'BLEU': 7.699406180106954e-06, 'brevity_penalty': 0.0019735783221064072, 'ratio': 0.13835263835263836, 'translation_length': 430, 'reference_length': 3108, 'precisions_0': 0.002320185614849188, 'precisions_1': 0.0030211480362537764, 'precisions_2': 0.004329004329004329, 'precisions_3': 0.007633587786259542}


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.29.2"
}



 69%|████████████████████████████▉             | 69/100 [04:36<02:04,  4.01s/it]


KeyboardInterrupt: 

In [22]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay         118G   22G   97G  19% /


In [23]:
!ls {current_experiment_path}

1684518421.931266   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518421.9377556  events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.7628372  events.out.tfevents.1684518911.c71a00d70d2c.1622.0
1684518911.768738   events.out.tfevents.1684518911.c71a00d70d2c.1622.2
checkpoint-500	    experiment_config.json


In [24]:
model.save_pretrained(current_experiment_path)

In [3]:
!ls /root/experiments/t2c_concode_220428_v34

1684518421.931266    checkpoint-500
1684518421.9377556   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518911.7628372   events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.768738    events.out.tfevents.1684518911.c71a00d70d2c.1622.0
adapter_config.json  events.out.tfevents.1684518911.c71a00d70d2c.1622.2
adapter_model.bin    experiment_config.json
